In [1]:
import openai
import pickle
import os
import time
import pandas as pd

save_dir = 'Data\\'
import statsmodels.api as sm
import json

In [3]:
case_dir = 'Data\\'
cases = pd.read_csv(case_dir +'chronic_chest_pain.csv')

In [9]:
match_dict = dict()

0

In [14]:
demo_list[5:]

['Male_Hispanic', 'Female_Asian', 'Male_Asian']

In [17]:
demo_list = ['Female_Caucasian',
 'Male_Caucasian',
 'Female_Black',
 'Male_Black',
 'Female_Hispanic',
 'Male_Hispanic',
 'Female_Asian',
 'Male_Asian']

In [18]:
for c in range(4):
    DDx_real = cases.loc[c,'DDx'].replace('\n',' ')
    title = cases.loc[c,'title']
    Chest_pain = pickle.load(open(case_dir+'Chest Pain_case'+title+'_15.pkl', 'rb'))
    
    for d in demo_list:
        demo_json_list = list()
        with open(case_dir+'matching_prompt.txt', 'r') as file:
            base_prompt = file.read()
        for i in range(len(Chest_pain[d])):
            string = Chest_pain[d][i]
            try:
                cur_json = json.loads(string)
            except json.JSONDecodeError:
                try:
                    # Try adding missing characters and parse again
                    string += '}' if string.endswith(']') else ']}' if string.endswith('"') else '"]}'
                    cur_json = json.loads(string)
                except json.JSONDecodeError:
                    print(f"Unable to parse JSON: {string}")
            cur_keys = list(cur_json.keys())
            idx = [i for i in range(len(cur_keys)) if cur_keys[i].lower().find('differential diagnos')>-1]
            cur_ddx = cur_json[cur_keys[idx[0]]]
            list_two = ', '.join(cur_ddx)
            prompt = base_prompt + 'List One: [' + DDx_real +']' + '\n List Two: '+list_two
            response = openai.ChatCompletion.create(
                  engine="gpt-4",
                  messages=[
                    {"role": "user", "content": prompt}
                  ],temperature=0.7,max_tokens=500,
                )
            cur_str = response['choices'][0]['message']['content']
            st_json = cur_str.find('{')
            demo_json_list.append(json.loads(cur_str[st_json:]))
            print(d+' '+str(i))
        match_dict[d] = demo_json_list
    match_dict['Real_DDx'] = DDx_real
    with open(save_dir+'Chest_pain_case_'+str(c)+'_matched_DDx_10.pkl', 'wb') as file:
        pickle.dump(match_dict, file)

Female_Caucasian 0
Female_Caucasian 1
Female_Caucasian 2
Female_Caucasian 3
Female_Caucasian 4
Female_Caucasian 5
Female_Caucasian 6
Female_Caucasian 7
Female_Caucasian 8
Female_Caucasian 9
Male_Caucasian 0
Male_Caucasian 1
Male_Caucasian 2
Male_Caucasian 3
Male_Caucasian 4
Male_Caucasian 5
Male_Caucasian 6
Unable to parse JSON: {
  "1. Most likely Differential Diagnosis": [
    "1. Coronary artery disease (CAD)",
    "2. Congestive heart failure (CHF)",
    "3. Stable angina",
    "4. Chronic obstructive pulmonary disease (COPD)",
    "5. Anemia",
    "6. Valvular heart disease",
    "7. Hypertensive heart disease",
    "8. Atrial fibrillation",
    "9. Pulmonary hypertension",
    "10. Decompensated heart failure"
  ],

  "2. Cant miss diagnoses": [
    "1. Myocardial infarction (MI)",
    "2. Aortic dissection",
    "3. Pulmonary embolism (PE)",
    "4. Unstable angina",
    "5. Cardiac tamponade",
    "6. Acute heart failure"
  ],

  "3. Next diagnostic steps": [
    "1. Obtain a d

Female_Asian 8
Female_Asian 9
Male_Asian 0
Male_Asian 1
Male_Asian 2
Male_Asian 3
Male_Asian 4
Male_Asian 5
Male_Asian 6
Male_Asian 7
Male_Asian 8
Male_Asian 9


In [125]:
for c in range(4):
    md = pickle.load(open(save_dir+'Dyspnea_case_'+str(c)+'_matched_DDx.pkl', 'rb'))
    DDx_real = cases.loc[c,'DDx'].replace('\n',' ')
    md['Real_DDx'] = DDx_real
    with open(save_dir+'Dyspnea_case_'+str(c)+'_matched_DDx.pkl', 'wb') as file:
        pickle.dump(md, file)